In [4]:
import glob
import pandas as pd

In [8]:
csv_files = glob.glob( '*/**.csv' )
print( result )

['subsid/prod_activations_logs.csv', 'subsid/tm_sales_1.csv', 'subsid/tm_sales_2.csv', 'subsid/tm_sales_3.csv']


In [220]:
activations = pd.read_csv(csv_files[0], sep=';', 
                          dtype={'SUBS_ID': str, 'PROD_ID': int}, 
                          dayfirst=True,
                          parse_dates=['START_DTTM', 'END_DTTM'])
sales = pd.DataFrame()
for csv in csv_files[1:]:
    df = pd.read_csv(csv, sep = ';')
    df = (
        df
         .query("SUBS_ID.isnull() == False")
         .assign(
                 SUBS_ID = 'id' + df.SUBS_ID.str.removeprefix("id")
                )
    )
    sales = pd.concat((sales, df), sort=True, ignore_index=True)

merged = activations.merge(sales, how = 'inner', on = ['SUBS_ID', 'PROD_ID'])

In [239]:
(
    merged
     .assign(difference = merged.END_DTTM - merged.START_DTTM,
             subs_clean = merged.SUBS_ID.str.removeprefix("id").astype(int))
     .query("difference.astype('timedelta64[m]') >= 5")
     .sort_values(by='subs_clean', ascending=False)
     .SUBS_ID.to_list()
)

['id7642700', 'id5416547', 'id4651830', 'id2185490']